# سیمینٹک کرنل

اس کوڈ نمونے میں، آپ [سیمینٹک کرنل](https://aka.ms/ai-agents-beginners/semantic-kernel) اے آئی فریم ورک کا استعمال کرتے ہوئے ایک بنیادی ایجنٹ بنائیں گے۔

اس نمونے کا مقصد آپ کو وہ مراحل دکھانا ہے جنہیں ہم بعد میں مختلف ایجنٹک پیٹرنز کو نافذ کرتے وقت اضافی کوڈ نمونوں میں استعمال کریں گے۔


## مطلوبہ پائتھن پیکجز درآمد کریں


In [ ]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## کلائنٹ بنانا

اس مثال میں، ہم [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) کا استعمال کریں گے تاکہ LLM تک رسائی حاصل کی جا سکے۔

`ai_model_id` کو `gpt-4o-mini` کے طور پر تعریف کیا گیا ہے۔ مختلف نتائج دیکھنے کے لیے ماڈل کو GitHub Models مارکیٹ پلیس میں دستیاب کسی دوسرے ماڈل میں تبدیل کرنے کی کوشش کریں۔

`Azure Inference SDK` استعمال کرنے کے لیے، جو GitHub Models کے `base_url` کے لیے استعمال ہوتا ہے، ہم Semantic Kernel کے اندر `OpenAIChatCompletion` کنیکٹر استعمال کریں گے۔ Semantic Kernel کو دوسرے ماڈل فراہم کنندگان کے لیے استعمال کرنے کے لیے دیگر [دستیاب کنیکٹرز](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) بھی موجود ہیں۔


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ایجنٹ بنانا

نیچے ہم `TravelAgent` نامی ایجنٹ بناتے ہیں۔

اس مثال میں، ہم بہت سادہ ہدایات استعمال کر رہے ہیں۔ آپ ان ہدایات کو تبدیل کر سکتے ہیں تاکہ دیکھ سکیں کہ ایجنٹ مختلف طریقے سے کیسے ردعمل دیتا ہے۔


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## ایجنٹس کو چلانا

اب ہم ایجنٹ کو چلانے کے لیے `ChatHistory` کو تعریف کریں گے اور اس میں `system_message` شامل کریں گے۔ ہم وہ `AGENT_INSTRUCTIONS` استعمال کریں گے جو ہم نے پہلے تعریف کیے تھے۔

جب یہ تعریف ہو جائیں، تو ہم ایک `user_inputs` بنائیں گے جو وہ پیغام ہوگا جو صارف ایجنٹ کو بھیج رہا ہے۔ اس مثال میں، ہم نے اس پیغام کو `Plan me a sunny vacation` پر سیٹ کیا ہے۔

آپ اس پیغام کو تبدیل کرنے کے لیے آزاد ہیں تاکہ دیکھ سکیں کہ ایجنٹ مختلف طریقے سے کیسے جواب دیتا ہے۔


In [ ]:
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہیں۔ اصل دستاویز، جو اس کی اصل زبان میں ہے، کو مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
